In [1]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNetV2
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
base_model=MobileNetV2(weights='imagenet', include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(512,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(512,activation='relu')(x) #dense layer 2
preds=Dense(3,activation='softmax')(x) #final layer with softmax activation

C:\Users\Giovanni\Miniconda3\envs\keras\lib\site-packages\keras_applications\mobilenet_v2.py:295: UserWarning: MobileNet shape is undefined. Weights for input shape(224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


In [3]:
model = Model(inputs=base_model.input, outputs=preds)

In [4]:
batch_size = 8

In [5]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                 validation_split=0.3) #included in our dependencies

In [6]:
train_generator = train_datagen.flow_from_directory('../data/',
                                                    target_size=(224,224),
                                                    color_mode='rgb',
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    subset='training',
                                                    shuffle=True)

Found 180 images belonging to 3 classes.


In [7]:
val_generator = train_datagen.flow_from_directory('../data/',
                                                  target_size=(224,224),
                                                  color_mode='rgb',
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  subset='validation',
                                                  shuffle=True)

Found 75 images belonging to 3 classes.


In [8]:
model.compile(optimizer=Adam(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
callbacks = [ModelCheckpoint('model_chkpt/weights.{epoch:02d}_{val_loss:.4f}_{val_acc:.4f}.h5')]

In [10]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch = train_generator.samples // batch_size,
                    validation_data=val_generator,
                    validation_steps = val_generator.samples // batch_size,
                    callbacks=callbacks,
                    epochs=20,
                    )

Epoch 1/20
22/22 [==============================] - 16s 730ms/step - loss: 0.7760 - acc: 0.6307 - val_loss: 0.6247 - val_acc: 0.7778
Epoch 2/20
22/22 [==============================] - 4s 175ms/step - loss: 0.3839 - acc: 0.8410 - val_loss: 0.4369 - val_acc: 0.8209
Epoch 3/20
22/22 [==============================] - 4s 160ms/step - loss: 0.1792 - acc: 0.9545 - val_loss: 0.3042 - val_acc: 0.8806
Epoch 4/20
22/22 [==============================] - 5s 207ms/step - loss: 0.1006 - acc: 0.9716 - val_loss: 0.2678 - val_acc: 0.8657
Epoch 5/20
22/22 [==============================] - 4s 196ms/step - loss: 0.0740 - acc: 0.9716 - val_loss: 0.2499 - val_acc: 0.8955
Epoch 6/20
22/22 [==============================] - 4s 185ms/step - loss: 0.0714 - acc: 0.9829 - val_loss: 0.2495 - val_acc: 0.9254
Epoch 7/20
22/22 [==============================] - 4s 204ms/step - loss: 0.0889 - acc: 0.9659 - val_loss: 0.1931 - val_acc: 0.9254
Epoch 8/20
22/22 [==============================] - 4s 172ms/step - loss: 0

In [11]:
best_model = keras.models.load_model('model_chkpt/weights.07_0.1931_0.9254.h5')

In [12]:
import tensorflowjs as tfjs

In [13]:
tfjs.converters.save_keras_model(best_model, 'model_output/')

In [39]:
model.predict(np.random.rand(1, 224, 224, 3))

array([[0.31375   , 0.00212419, 0.68412584]], dtype=float32)